<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Build-function-for-building-cascade" data-toc-modified-id="Build-function-for-building-cascade-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Build function for building cascade</a></span><ul class="toc-item"><li><span><a href="#Test-shortest-path-function" data-toc-modified-id="Test-shortest-path-function-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Test shortest path function</a></span></li></ul></li><li><span><a href="#Import-dataset" data-toc-modified-id="Import-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import dataset</a></span><ul class="toc-item"><li><span><a href="#Store-cascade-data" data-toc-modified-id="Store-cascade-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Store cascade data</a></span></li><li><span><a href="#Store-cascade-seed-data" data-toc-modified-id="Store-cascade-seed-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Store cascade seed data</a></span></li></ul></li><li><span><a href="#Model" data-toc-modified-id="Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Criticize-model-compared-to-NETRATE-results" data-toc-modified-id="Criticize-model-compared-to-NETRATE-results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Criticize model compared to NETRATE results</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#pathEM-Algorithm" data-toc-modified-id="pathEM-Algorithm-4.0.1"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span>pathEM Algorithm</a></span></li></ul></li></ul></li></ul></div>

In [1]:
import tensorflow as tf
import edward as ed
import numpy as np

from edward.models import Beta, Exponential, Gamma, Normal

In [2]:
sess = ed.get_session()

# Build function for building cascade

In [3]:
def build_cascade(time, seed, T):
    # Store number of nodes
    n = time.shape[0]

    # Transpose times and reduce minimum
    times_T = tf.minimum(tf.transpose(time),T)

    # Initialize transmission times to be max time except for seed node
    transmission = tf.ones(n)*T
    transmission = tf.subtract(transmission,tf.one_hot(seed, n)*T)

    
    # Continually update transmissions
    for _ in range(n):

        # Tile transmission
        transmission_tiled = tf.reshape(tf.tile(transmission,[n]),[n,n])

        # Add transposed times and tiled transmissions
        potential_transmission = tf.add(transmission_tiled,times_T)

        # Find minimum path from all new 
        potential_transmission_row = tf.reduce_min(potential_transmission, reduction_indices=[1])

        # Concatenate previous transmission and potential new transmission
        potential_transmission_stack = tf.stack([transmission,potential_transmission_row],axis=0)

        # Take the minimum of the original transmission and the potential new transmission
        transmission = tf.reduce_min(potential_transmission_stack, reduction_indices=[0])

    return transmission

In [4]:
def build_cascade_series(time, seeds, T):

    transmission_stack = tf.map_fn(lambda x: build_cascade(time, x, T), seeds, dtype=tf.double)
    return tf.cast(transmission_stack,tf.float64)

## Test shortest path function

In [53]:
alpha = tf.convert_to_tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],tf.float32)
tau = Exponential(alpha)

sess.run(build_cascade(tau, 2, 10))

array([10., 10.,  0., 10., 10., 10., 10., 10., 10., 10.], dtype=float32)

# Import dataset

In [6]:
with open('kronecker-core-periphery-n1024-h10-r0_01-0_25-1000-cascades.txt','r') as f:
    
    # Store number of nodes
    numNodes = -1
    while True:
        if f.readline() == "\n":
            break
        numNodes+=1

    # Collect cascades into list
    v = []
    for line in f.readlines():
        v.append([float(l) for l in line.rstrip('\n').split(",")])

## Store cascade data

In [91]:
np_cascades = np.ones((len(v),numNodes),np.float32)*100
for row, cascade in enumerate(v):  
    c_nodes = [int(cascade[i*2]) for i in range(len(cascade)//2)]
    c_times = [cascade[i*2+1] for i in range(len(cascade)//2)]

    for col in range(len(c_nodes)):
        np_cascades[row][c_nodes[col]]=c_times[col]

## Store cascade seed data

In [92]:
np_seeds = np.array([cascade[0] for cascade in v], dtype = np.int)

# Model

In [93]:
# Time horizon of cascade
T = 100

# Gamma(1,1) priors on alpha
ed_a = Gamma(tf.ones([numNodes,numNodes]),tf.ones([numNodes,numNodes]))

# Exponentiate alpha to get times
ed_tau = Exponential(ed_a)

# Placeholder for seed input data
seed_ph = tf.placeholder(tf.int64, [np_seeds.shape[0],])

# Build series of C cascades
cascade_ph = build_cascade_series(ed_tau, seed_ph, T)

In [64]:
# q_a = Normal(loc=tf.get_variable("q_a/loc", [numNodes, numNodes]),
#             scale=tf.nn.softplus(tf.get_variable("q_a/scale", [numNodes, numNodes])))

In [96]:
data = {cascade_ph: np_cascades,
        seed_ph: np_seeds}
inference = ed.KLqp({ed_a: q_a}, data)

/Users/brianallen131/src/edward/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


In [95]:
data = {cascade_ph: np_cascades,
# data = {cascade_ph: np.repeat(np_cascades[0].reshape(1,1023),1001, axis=0),
        seed_ph: np_seeds}
inference = ed.MAP([ed_a], data)

/Users/brianallen131/src/edward/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


In [97]:
tf.global_variables_initializer().run()

# optimizer = tf.train.AdamOptimizer(0.01, epsilon=1.0)
# inference.initialize(optimizer=optimizer)

inference.run(n_iter=2000)
inference.finalize()

2000/2000 [100%] ██████████████████████████████ Elapsed: 131s | Loss: 13998.625


In [105]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)


sess.run(tf.get_collection('variables', scope='optimizer_4/q_a/loc/Adam'))[0][753][261]

0.20594983

In [ ]:
optimized_alpha = sess.run(inference.latent_vars)

In [ ]:
list(optimized_alpha.values())[0][753].argsort()[::-1]

In [ ]:
loc_sample = q_a.sample(100)
mu_sample = ed_a.sample(100)

In [100]:
def printCascade(cascade):
    print("order\t node\t time")
    print("-----\t ----\t ----")
    for i in range(len(cascade)//2):
        print('{:5d}\t {:4d}\t {:0.2f}'.format(i+1,int(cascade[i*2]), cascade[i*2+1]))

printCascade(v[0])

order	 node	 time
-----	 ----	 ----
    1	  753	 0.00
    2	  261	 1.35
    3	  704	 3.97
    4	  145	 4.90
    5	   73	 5.93
    6	  286	 6.63
    7	  556	 7.84
    8	   80	 8.14
    9	  458	 8.78
   10	    8	 8.79
   11	  467	 8.90
   12	  554	 8.91
   13	   53	 9.24
   14	    1	 9.45
   15	  584	 9.59
   16	  843	 9.60
   17	  116	 9.78
   18	  130	 9.82
   19	  545	 9.83
   20	  579	 9.97
   21	  535	 9.98


# Criticize model compared to NETRATE results

In [ ]:
import matplotlib.pyplot as plt
# plt.imshow(list(optimized_alpha.values())[0], cmap='hot')

In [29]:
netrate = np.loadtxt(open("netrate alpha.txt", "rb"), delimiter=",")

In [ ]:
plt.imshow(netrate,cmap='hot')

In [ ]:
netrate[753][261]

In [ ]:
np_cascades.shape

In [ ]:
plt.hist(np.ravel(netrate),bins=50)

In [45]:
netrate[753][217]

0.0

### pathEM Algorithm

** Shortest Path Based Decision Making Using Probabilistic Inference **

In [46]:
alpha = tf.convert_to_tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],tf.float32)
tau = Exponential(alpha)


def pathEM(time):
    x_ij = tf.ones(time.shape)/2
    xtilde_ij = tf.ones(time.shape)/2
    lambda_i = tf.zeros(time.shape)
    lambda_ij = tf.ones(time.shape)
    
    for _ in range(10):
        g_lambda = 
    
pathEM(tau)

In [106]:
np_cascades[0]

array([100.      ,   9.451033, 100.      , ..., 100.      , 100.      ,
       100.      ], dtype=float32)